# This notebook converts match data into individual player data.

## import, csv, json

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
folder = "EUW"
region = "eu"

In [3]:
df = pd.read_csv('data/{}/{}clean.csv'.format(folder,region),low_memory=False)
print(df.shape)

(14263, 1165)


## regex + participants

In [4]:
# function to edit desired participant columns

def strippart(num):
    res = []
    for name in df.columns:
        if re.search('^participants_{}_'.format(num), name) != None:
            res += [name]
    res = [re.sub('^participants_{}_'.format(num), '', x) for x in res]
    return res

In [5]:
# use function for each participant column 0-9

stripped = [set(strippart(x)) for x in range(0,10)]
print([len(x) for x in stripped])

[106, 105, 105, 105, 105, 104, 104, 104, 104, 104]


In [6]:
# drop uncommon elements
common = list(set.intersection(*stripped))
print(len(common))

104


In [7]:
# non participant specific data to keep
start = list(df.columns[:5])

# non participant team based data to keep
team = []
for name in df.columns:
    if re.search('^team', name) != None:
        team += [name]

# non participant player based data to keep
playercol = ['player_platformId', 'player_accountId', 'player_summonerName', 'player_summonerId', 
          'currentAccountId', 'player_matchHistoryUri', 'player_profileIcon']

# participant based data 
partid = []
for name in df.columns:
    if re.search('^participantIdentities', name) != None:
        partid += [name]

# function to construct df based on participant
def isolate(num):
    participant = ['participants_{}_{}'.format(num,x) for x in common]
    player = [x for x in partid if str(num) in x]
    res = df.copy()[start + team + participant + player]
    newcol = start + team + common + playercol
    res.columns = newcol
    res['participant_number'] = num
    return res

In [8]:
# loop the function
i = [isolate(x) for x in range(0,10)] 

In [9]:
# create df

participants = pd.concat(i)
participants

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,stats_perk4Var2,stats_trueDamageDealt,player_platformId,player_accountId,player_summonerName,player_summonerId,currentAccountId,player_matchHistoryUri,player_profileIcon,participant_number
0,4681072641,1593268978835,1299,420,10.13.326.4870,0,0,0,0,0,...,0,0,EUW1,uHf7lxqD_AHXgcQ7w8BsYZJkLQ3A32NcvNbEC9tL76_ctoU,Keny keczułoki,rkbgzOBWWOJHv8pp31OtIgvhPCsAtjAQgzyPDdUMEcYAG-M,uHf7lxqD_AHXgcQ7w8BsYZJkLQ3A32NcvNbEC9tL76_ctoU,/v1/stats/player_history/EUW1/227273950,4495,0
1,4707844099,1594758284675,1438,420,10.14.327.8505,0,0,0,0,0,...,1771,1845,EUW1,R8V8agqSRpOWzXNKCQFDOe7uYYwPSRR16dTyCbHVzMRxjQ,backstreetb0ys,6lFlyTmRE9dMVT6dUaS_s_Q1I5hQ3xmkRKEtirplOfwxy0Q,R8V8agqSRpOWzXNKCQFDOe7uYYwPSRR16dTyCbHVzMRxjQ,/v1/stats/player_history/EUW1/32430502,4568,0
2,4679991311,1593201304458,1826,420,10.13.326.4870,1,1,1,1,1,...,500,2694,EUW1,7UwmsxJcxwBlpS4R5h9eN2PWuiqvi1sgKKuNTvSFXLiDuq...,losing lp,YEFgyA2pwHA-2pjUGY1OLpQkb46Fzh6szOrBbuHuyEokT5f4,7UwmsxJcxwBlpS4R5h9eN2PWuiqvi1sgKKuNTvSFXLiDuq...,/v1/stats/player_history/EUW1/2342509655527872,4571,0
3,4648730648,1591566971144,1523,420,10.11.322.2991,0,1,0,0,0,...,4,8536,EUW1,7vec5bkkSJVMm1cmPzXA-eHe21OMg78orn0ud4HeZ_jYcbY,Durum Enjoyer,O-kkiXorXHMhZD-oX9DsSkX9efVvXb_5YgZYD_ZoOuBfAIY,7vec5bkkSJVMm1cmPzXA-eHe21OMg78orn0ud4HeZ_jYcbY,/v1/stats/player_history/EUW1/224568594,22,0
4,4632641560,1590840449379,1908,420,10.11.322.2991,1,1,0,1,1,...,500,5671,EUN1,nB-Ngx_XdTR7JMljlZ3hPMcFiysKch-ldAcKQrDmEP8t7V0,Barcodee,lWz-CSIcQ6j1nuoSKbDNqmgBQK4qHJhH9ydCVg6RaPCbOIM,0KtEdshrlLyUEsIjvb5UFjJPvrKNlMcKvsVNP4qauMwdcfg,/v1/stats/player_history/EUN1/205884979,4574,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14258,4656414258,1591957338230,1417,420,10.12.324.5925,0,0,0,0,0,...,0,6495,EUW1,rvXKTSPff87zkktIOApiy4bGNuDkw-Lt82QdFlPReglFgkg,Fallen Azazel,zOznRvcKMWg5vcaEt-dXYOSakwPi1mY5LKAOOfCoBZay9X8,rvXKTSPff87zkktIOApiy4bGNuDkw-Lt82QdFlPReglFgkg,/v1/stats/player_history/EUW1/220921936,657,9
14259,4682563131,1593355545520,1479,420,10.13.326.4870,1,1,1,1,0,...,140,11154,EUW1,3yC6UhjYiy6c4hLALZZ3ZVHEndhpZaIQTU0dJVdQuVRahQ,RekSaiyajin,4JhPlXQqSQVmgev0Vk3qkKuRao2uUy-BlBcLyUsothoZ7yA,3yC6UhjYiy6c4hLALZZ3ZVHEndhpZaIQTU0dJVdQuVRahQ,/v1/stats/player_history/EUW1/29103829,4515,9
14260,4684594750,1593456408458,1970,440,10.13.326.4870,1,0,0,1,1,...,757,882,EUW1,g8dNb1c34aPg0zXufq90xX66HdIFCIF7HAdKb9XmgSkqAG8,Shunporìna,gpcJZayACNDJK0Zp_hkxQQ1WbRpq9_Yg7QDTvvOnRAY8TCE,g8dNb1c34aPg0zXufq90xX66HdIFCIF7HAdKb9XmgSkqAG8,/v1/stats/player_history/EUW1/226356513,3785,9
14261,4675157584,1592935405722,1610,420,10.12.325.6222,1,1,1,1,1,...,0,11367,EUN1,QtDjDzYVBdup1lAaInQxvOnQZrqYBtl7K-bPivkuFQKz0YA,SPX Nafkelah,JfPnSn3rT1avRa-uA-hzevX49A5UlaDv9ITEI2JIemqQ6-6s,u7vcYcW7UTmkit3IvB1gqPmPmBjnq36AM57onjdBk_h7Tx...,/v1/stats/player_history/EUN1/208523511,4317,9


In [10]:
# export

#compression_opts = dict(method='zip',
#                        archive_name='{}player.csv'.format(region))  
#participants.to_csv('{}player.zip'.format(region), index=False, compression=compression_opts)